In [1]:
!pip install torch transformers datasets
!pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from google.colab import userdata
import os
os.environ['GIT_TOKEN'] = secret_value = userdata.get('GIT_TOKEN')

In [2]:
!git clone https://$GIT_TOKEN@github.com/Abhishek-P/disrpt25-task.git

Cloning into 'disrpt25-task'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 64 (delta 36), reused 32 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 39.25 KiB | 608.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
%cd '/content/disrpt25-task'
!ls
!pip install conllu

/content/disrpt25-task
disrptdata.py  mapping_disrpt25.json  requirements.txt	util.py
LICENSE        README.MD	      run_model.py


In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import sys
sys.path.append('/content/disrpt25-task')

Mounted at /content/drive/


In [5]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import torch
import torch.nn as nn
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import classification_report, accuracy_score, f1_score
import argparse
import numpy as np
import csv

In [6]:
import disrptdata
disrptdata.DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/sample_data"

In [7]:
# === load dataset ===
"""
experiment with two languages: Chinese and English
"""
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/sample_data"

# load and combine datasets
zho = disrptdata.get_dataset("zho.rst.gcdt")
eng = disrptdata.get_dataset("eng.erst.gum")
combined = disrptdata.get_combined_dataset()
print("Train examples:", combined["train"].num_rows)
print("Dev examples:", combined["dev"].num_rows)

train_dataset = combined['train']
dev_dataset = combined['dev']
train_dataset = train_dataset.class_encode_column('label')
dev_dataset   = dev_dataset.class_encode_column('label')

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")

# preprocess data
def preprocess(example):
    # encode language, framework, corpus name, relation direction as tokens to the input
    # ref: https://gitlab.irit.fr/melodi/andiamo/relation-classification-features/-/blob/main/utils.py?ref_type=heads

    meta = (f"[LANG:{example['lang']}] "
            f"[FW:{example['framework']}] "
            f"[CORP:{example['corpus']}]")

    if example['direction'] == '1>2':
        span = f"Arg1: }} {example['u1']} > [SEP] Arg2: {example['u2']}"
    elif example['direction'] == '1<2':
        span = f"Arg1: {example['u1']} < Arg2: {example['u2']} {{"
    else:
        span = f"Arg1: {example['u1']} [SEP] Arg2: {example['u2']}"

    text = f"Classify: {meta} {span}"

    # text = f"Classify: {meta} Arg1: {example['u1']} Arg2: {example['u2']}"
    encoded = tokenizer(text, padding="max_length", truncation=True, max_length=512)
    encoded["label"] = example["label"]
    return encoded

# will use batch when dataset size scales up
train_tokenized = train_dataset.map(preprocess, batched=False)
dev_tokenized   = dev_dataset.map(preprocess, batched=False)

train_tokenized.set_format('torch', columns=["input_ids", "attention_mask", "label"])
dev_tokenized.set_format('torch', columns=["input_ids", "attention_mask", "label"])

Found the following datasets in the data directory:
Train examples: 4527
Dev examples: 4714


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Casting to class labels:   0%|          | 0/4527 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4714 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/4527 [00:00<?, ? examples/s]

Map:   0%|          | 0/4714 [00:00<?, ? examples/s]

In [10]:
class MT5Classifier(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.encoder = MT5ForConditionalGeneration.from_pretrained("google/mt5-small").get_encoder()
        hidden_size = self.encoder.config.d_model
        # classifier head
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state.mean(dim=1)
        logits = self.classifier(pooled)
        loss = self.loss_fct(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

In [11]:
num_labels = train_tokenized.features["label"].num_classes
model = MT5Classifier(num_labels=num_labels)

In [12]:
# === training setup ===
use_cuda = True
device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mt5_LCFD/mt5_classifier_LCFD_2.results",
    overwrite_output_dir=False,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    auto_find_batch_size=True,
)


data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    # get label names from the feature
    label_names = train_tokenized.features["label"].names

    report = classification_report(labels, preds, target_names=label_names)
    print("\n=== Classification Report ===")
    print(report)
    return {"accuracy": acc, "f1": f1}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

batch = next(iter(trainer.get_train_dataloader()))
print("Batch input_ids shape:", batch["input_ids"].shape)

/tmp/ipython-input-12-2979332787.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Batch input_ids shape: torch.Size([2, 512])


In [13]:
trainer.train()
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jw2175 (jw2175-georgetown-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.407300,2.425670,0.255834,0.151081
2,2.289000,2.380815,0.256046,0.151078



=== Classification Report ===
               precision    recall  f1-score   support

  alternation       0.00      0.00      0.00        36
  attribution       0.00      0.00      0.00       318
       causal       0.00      0.00      0.00       238
      comment       0.00      0.00      0.00       180
   concession       0.00      0.00      0.00       189
    condition       0.00      0.00      0.00        93
  conjunction       0.41      0.31      0.35       777
     contrast       0.00      0.00      0.00       199
  elaboration       0.26      0.78      0.38       879
  explanation       0.00      0.00      0.00       382
        frame       0.00      0.00      0.00       235
         mode       0.00      0.00      0.00       104
 organization       0.19      0.88      0.32       315
      purpose       0.00      0.00      0.00       161
        query       0.00      0.00      0.00        79
reformulation       0.00      0.00      0.00       156
     temporal       0.00      0.0

...


=== Classification Report ===
               precision    recall  f1-score   support

  alternation       0.00      0.00      0.00        36
  attribution       0.00      0.00      0.00       318
       causal       0.00      0.00      0.00       238
      comment       0.00      0.00      0.00       180
   concession       0.00      0.00      0.00       189
    condition       0.00      0.00      0.00        93
  conjunction       0.40      0.31      0.35       777
     contrast       0.00      0.00      0.00       199
  elaboration       0.26      0.78      0.39       879
  explanation       0.00      0.00      0.00       382
        frame       0.00      0.00      0.00       235
         mode       0.00      0.00      0.00       104
 organization       0.19      0.88      0.32       315
      purpose       0.00      0.00      0.00       161
        query       0.00      0.00      0.00        79
reformulation       0.00      0.00      0.00       156
     temporal       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



=== Classification Report ===
               precision    recall  f1-score   support

  alternation       0.00      0.00      0.00        36
  attribution       0.00      0.00      0.00       318
       causal       0.00      0.00      0.00       238
      comment       0.00      0.00      0.00       180
   concession       0.00      0.00      0.00       189
    condition       0.00      0.00      0.00        93
  conjunction       0.40      0.31      0.35       777
     contrast       0.00      0.00      0.00       199
  elaboration       0.26      0.78      0.39       879
  explanation       0.00      0.00      0.00       382
        frame       0.00      0.00      0.00       235
         mode       0.00      0.00      0.00       104
 organization       0.19      0.88      0.32       315
      purpose       0.00      0.00      0.00       161
        query       0.00      0.00      0.00        79
reformulation       0.00      0.00      0.00       156
     temporal       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.380814790725708,
 'eval_accuracy': 0.256045820958846,
 'eval_f1': 0.15107773268623845,
 'eval_runtime': 54.3615,
 'eval_samples_per_second': 86.716,
 'eval_steps_per_second': 43.358,
 'epoch': 2.0}

In [ ]:
# === error analysis ===
import csv
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Get predictions on tokenized dev set
pred_out = trainer.predict(dev_tokenized)
logits = pred_out.predictions
labels = pred_out.label_ids
preds = np.argmax(logits, axis=1)
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()

# Use label names from the dataset
label_names = dev_dataset.features["label"].names

# Use original dev_dataset (not tokenized) for u1 and u2
formatted_texts = [
    f"Arg1: {ex['u1']} | Arg2: {ex['u2']}" for ex in dev_dataset
]

# Extract misclassified examples
mis = [
    (text, label_names[true], label_names[pred], probs[i][pred])
    for i, (text, true, pred) in enumerate(zip(formatted_texts, labels, preds))
    if true != pred
]

# Save to Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

out_path = "/content/drive/MyDrive/mt5_vanilla/misclassifications_LCFD.csv"

with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "true_label", "pred_label", "confidence"])
    writer.writerows(mis)

print(f"✅ Saved {len(mis)} misclassified examples to:\n{out_path}")